In [1]:
from ast import literal_eval
from pathlib import Path
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

DATA_DIR = Path().resolve().parent / 'data'

## Загрузка данных

In [2]:
df = pd.read_csv(DATA_DIR / 'Spb_flats_prices.csv')
df.head()

,Underground Times,"Flat Status (1 - handed over, 0 - not handed over)",Flat Type,Total Area (m^2),Living Area (m^2),Kitchen Area (m^2),Floor,Year of building,Price (Rubles)
0,"{'Underground name': ['Балтийская', 'Василеост...",1,4-комн,125.50,75.8,22.9,4,1876,24500000
1,"{'Underground name': ['Московская', 'Бухарестс...",1,2-комн,41.50,26.4,6.0,3,1964,6300000
2,"{'Underground name': ['Василеостровская', 'Спо...",0,3-комн,80.39,44.9,10.2,3,2025,30061520
3,"{'Underground name': ['Улица Дыбенко', 'Проспе...",1,2-комн,65.10,27.0,19.0,4,1983,11500000
4,"{'Underground name': ['Девяткино', 'Граждански...",1,1-комн,38.50,16.6,10.5,10,2015,5599000


In [3]:
df.columns = ['metro_info',
             'flat_status',
             'flat_type',
             'total_area_m2',
             'living_area_m2',
             'kitchen_area_m2',
             'floor',
             'year_of_bulding',
             'price']

In [4]:
df.head()

,metro_info,flat_status,flat_type,total_area_m2,living_area_m2,kitchen_area_m2,floor,year_of_bulding,price
0,"{'Underground name': ['Балтийская', 'Василеост...",1,4-комн,125.50,75.8,22.9,4,1876,24500000
1,"{'Underground name': ['Московская', 'Бухарестс...",1,2-комн,41.50,26.4,6.0,3,1964,6300000
2,"{'Underground name': ['Василеостровская', 'Спо...",0,3-комн,80.39,44.9,10.2,3,2025,30061520
3,"{'Underground name': ['Улица Дыбенко', 'Проспе...",1,2-комн,65.10,27.0,19.0,4,1983,11500000
4,"{'Underground name': ['Девяткино', 'Граждански...",1,1-комн,38.50,16.6,10.5,10,2015,5599000


In [5]:
def nearest_metro_station(json):
    stations_dict = literal_eval(json)
    nearest_station = stations_dict['Underground name'][0]
    time_to_get = stations_dict['Underground time'][0]
    way_to_get_to = stations_dict['Transport type'][0]
    return [nearest_station, time_to_get, way_to_get_to]

In [6]:
df['metro_info'] = df['metro_info'].apply(lambda x: nearest_metro_station(x))

In [7]:
df['metro_info']

0                          [Балтийская, 5, car]
1                          [Московская, 6, car]
2                  [Василеостровская, 11, legs]
3                       [Улица Дыбенко, 7, car]
4                           [Девяткино, 6, car]
                         ...                   
1075                       [Спортивная, 6, car]
1076    [Площадь Александра Невского, 14, legs]
1077               [Василеостровская, 11, legs]
1078                         [Беговая, 13, car]
1079                      [Московская, 21, car]
Name: metro_info, Length: 1080, dtype: object

In [8]:
df[['metro_station', 'minutes_to_metro', 'transfer_type']] = pd.DataFrame(df['metro_info'].tolist(), index=df.index)
df.drop('metro_info', axis=1, inplace=True)

In [9]:
df.head()

,flat_status,flat_type,total_area_m2,living_area_m2,kitchen_area_m2,floor,year_of_bulding,price,metro_station,minutes_to_metro,transfer_type
0,1,4-комн,125.50,75.8,22.9,4,1876,24500000,Балтийская,5,car
1,1,2-комн,41.50,26.4,6.0,3,1964,6300000,Московская,6,car
2,0,3-комн,80.39,44.9,10.2,3,2025,30061520,Василеостровская,11,legs
3,1,2-комн,65.10,27.0,19.0,4,1983,11500000,Улица Дыбенко,7,car
4,1,1-комн,38.50,16.6,10.5,10,2015,5599000,Девяткино,6,car


In [11]:
df.duplicated().sum()

np.int64(82)